Montar Google Drive

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
# Cambia a la carpeta raíz del proyecto
%cd /content/drive/MyDrive/Proyecto_Analisis_de_Sentimientos_G4
!pwd && ls -la # Muestra la ruta actual y la lista de archivos


Mounted at /content/drive
/content/drive/MyDrive/Proyecto_Analisis_de_Sentimientos_G4
/content/drive/MyDrive/Proyecto_Analisis_de_Sentimientos_G4
total 30
drwx------ 2 root root 4096 Aug 15 06:59 api
-rw------- 1 root root 1086 Aug 15 08:16 api.log
drwx------ 2 root root 4096 Aug 15 08:16 dashboard
-rw------- 1 root root  273 Aug 15 07:54 dash.log
drwx------ 2 root root 4096 Aug 13 01:12 data
drwx------ 2 root root 4096 Aug 13 03:32 docs
drwx------ 2 root root 4096 Aug 13 03:32 models
drwx------ 2 root root 4096 Aug 13 03:32 notebooks
drwx------ 2 root root 4096 Aug 15 06:23 src


Verificar carpetas

In [2]:
from src.utils.config_rutas import reports_dir # Importa la ruta de la carpeta de reportes
print("reports_dir ->", reports_dir.resolve())
!ls -la "{reports_dir}" # Muestra el contenido de la carpeta de reportes


reports_dir -> /content/drive/MyDrive/Proyecto_Analisis_de_Sentimientos_G4/docs/reports
total 952
-rw------- 1 root root  50126 Aug 15 07:54 alerts_log.csv
-rw------- 1 root root 924461 Aug 15 07:54 results_log.csv


Instalar dependencias

In [3]:
!pip -q install fastapi "uvicorn[standard]" pydantic requests pandas streamlit altair wordcloud pyngrok


Iniciar API (FastAPI)

In [4]:
# limpia: mata uvicorn y libera puerto 8000
!pkill -f uvicorn || true
!fuser -n tcp 8000 -k 2>/dev/null || true

# arranca uvicorn (app dentro de api/api.py)
!PYTHONPATH=. nohup uvicorn api.api:app --host 0.0.0.0 --port 8000 --reload > api.log 2>&1 &
!sleep 2 # Espera 2 segundos para que arranque
!tail -n 80 api.log # Muestra las últimas líneas del log


^C
INFO:     Will watch for changes in these directories: ['/content/drive/MyDrive/Proyecto_Analisis_de_Sentimientos_G4']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [104300] using WatchFiles


Verificar API

In [5]:
import time, requests
BASE = "http://127.0.0.1:8000"

ok = False
for _ in range(10):  # Intenta hasta 10 veces
    try:
        print(requests.get(f"{BASE}/health", timeout=3).json())
        ok = True
        break
    except Exception:
        time.sleep(1) #Se espera cada 1 segundo y vuelve a intentar

print("API_OK:", ok)


{'status': 'ok'}
API_OK: True




Procesar CSV con /bath



In [6]:
import requests, json

payload = {
    "csv_path": "/content/drive/MyDrive/Proyecto_Analisis_de_Sentimientos_G4/data/raw/amazon_reviews_limpio_balanceado.csv",  # <-- ajusta si usas otro
    "text_col": "text", # Columna del CSV que contiene el texto a analizar
    "max_rows": 500     # Número máximo de filas a procesar
}
try:
    r = requests.post(f"{BASE}/batch", json=payload, timeout=120)
    print("status:", r.status_code)
    print(r.text[:600])  #Muestra los primeros 600 caracteres de la respuest
except Exception as e:
    print("ERROR:", e)


status: 200
{"ok":true,"processed":500,"neg_high_alerts":41,"results_csv":"/content/drive/MyDrive/Proyecto_Analisis_de_Sentimientos_G4/docs/reports/results_log.csv","alerts_csv":"/content/drive/MyDrive/Proyecto_Analisis_de_Sentimientos_G4/docs/reports/alerts_log.csv"}


Iniciar con el dashboard

In [7]:
# desde la raíz del proyecto
%cd /content/drive/MyDrive/Proyecto_Analisis_de_Sentimientos_G4

# cierra streamlit previo (si lo hubiera)
!pkill -f streamlit || true

# arranca el dashboard
%cd dashboard
!nohup streamlit run dashboard_app.py --server.headless true --server.port 8501 > ../dash.log 2>&1 &

# muestra el log
!sleep 3
!tail -n 60 ../dash.log


/content/drive/MyDrive/Proyecto_Analisis_de_Sentimientos_G4
^C
/content/drive/MyDrive/Proyecto_Analisis_de_Sentimientos_G4/dashboard



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.135.81.158:8501



Exponer el dashboard con ngrok

In [8]:
# Asegura que Streamlit está corriendo en 8501
%cd /content/drive/MyDrive/Proyecto_Analisis_de_Sentimientos_G4/dashboard
!pkill -f streamlit || true
!nohup streamlit run dashboard_app.py --server.headless true --server.address 0.0.0.0 --server.port 8501 > dash.log 2>&1 &
!sleep 3
!tail -n 20 dash.log
!curl -sSf http://127.0.0.1:8501/ | head -n 3


/content/drive/MyDrive/Proyecto_Analisis_de_Sentimientos_G4/dashboard
^C



  You can now view your Streamlit app in your browser.

  URL: http://0.0.0.0:8501

<!--
 Copyright (c) Streamlit Inc. (2018-2022) Snowflake Inc. (2022-2025)



In [9]:
# 2) Abre túnel ngrok SIN headers
!pip -q install pyngrok
from pyngrok import ngrok

ngrok.set_auth_token("31IvUnxPlx03CNgeXZV9klNJyqb_7VvQMQXFhbThH8T11nyt2")   # pega tu token
# cierra túneles previos
for t in ngrok.get_tunnels():
    ngrok.disconnect(t.public_url)

public_url = ngrok.connect(8501, bind_tls=True)  # <- SIN request_header
public_url


<NgrokTunnel: "https://a58cf4815eaa.ngrok-free.app" -> "http://localhost:8501">



---

